<a href="https://colab.research.google.com/github/AnnaZhuravleva/HSE/blob/master/course%204/sem03/Sem03_semantics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymorphy2

     |████████████████████████████████| 51kB 2.5MB/s 
     |████████████████████████████████| 7.1MB 10.0MB/s 


In [14]:
from google.colab import drive
import sys
drive.mount('/content/drive')
elmo_path = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/elmo'
project_path = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/'
sys.path.append(project_path)
import time
import gensim
import sklearn
from sklearn.metrics.pairwise import cosine_similarity
from elmo_helpers import tokenize, get_elmo_vectors, load_elmo_embeddings
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
from BM25 import Data, BM25
import tensorflow as tf
from time import time
# import urllib.request
import pandas as pd
import numpy as np
import collections
import pymorphy2
import logging
import nltk
import sys
import csv
import re
import os


pmm = pymorphy2.MorphAnalyzer()
root = logging.getLogger()
root.setLevel(logging.DEBUG)
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.INFO)
root.addHandler(handler)
nltk.download('punkt')

Mounted at /content/drive
Loading dictionaries from /usr/local/lib/python3.6/dist-packages/pymorphy2_dicts/data
Loading dictionaries from /usr/local/lib/python3.6/dist-packages/pymorphy2_dicts/data
format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
format: 2.4, revision: 393442, updated: 2015-01-17T16:03:56.586168
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [0]:
collection = 'content/drive/My Drive/Colab Notebooks/infosearch/sem3/quora_question_pairs_rus.csv'

# **FastText Search**



In [0]:
def build_test_data(corpus, collection):
  data = pd.read_csv(corpus)
  collection = pd.read_csv(collection)
  top = len(data['question1'])
  test_data = []
  for idx in range(top):
    if data['is_duplicate'][idx] == 1:
      test_data.append((idx, data['question1'][idx], collection['question2'][idx]))
  return test_data


In [0]:
class DataSet:

  def __init__(self, corpus):
    self.corpus = pd.read_csv(corpus)
    self.collection = self.corpus['question2']

  def simple_preproc(self, query):
      return [pmm.normal_forms(word)[0] for word in nltk.word_tokenize(query)]

  def normalized_collection(self):
      a = 0
      with open('collection.csv', 'w', newline='', encoding='utf8') as w:
          writer = csv.writer(w, delimiter=',')
          writer.writerow(['question1', 'question2', 'is_duplicate',
                           'question1_n', 'question2_n',])
          for j, x, z in zip(self.corpus['question1'],
                             self.corpus['question2'],
                             self.corpus['is_duplicate']):
              try:
                  line1 = self.simple_preproc(j)
                  line1 = ' '.join(line1)
              except TypeError:
                  line1 = ' '
              try:
                  line2 = self.simple_preproc(x)
                  line2 = ' '.join(line2)
              except TypeError:
                  line2 = ' '
              writer.writerow([j, x, z, line1, line2])
              print(a)
              a += 1
      


class FastTextSearch(DataSet):

    def __init__(self, model, corpus):
        logging.info('====\nStart FastText initializing!')
        DataSet.__init__(self, corpus=corpus)
        self.model = gensim.models.KeyedVectors.load(model)
        self.test_data = []
        #if os.path.isfile('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/models/fasttext.npy'):
        self.matrix = np.load('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/models/fasttext.npy')
        # else:
        #    self.matrix = self.index_fasttext()
        logging.info('FastText initialized!')
        
    def build_vec(self, query):
        if isinstance(query, str):
            query = self.simple_preproc(query)
        lemmas_vectors = np.zeros((len(query), self.model.vector_size))
        vec = np.zeros((self.model.vector_size,))
        for idx, lemma in enumerate(query):
            if lemma in self.model.vocab:
                lemmas_vectors[idx] = self.model.wv[lemma]
            else:
                pass
        if lemmas_vectors.shape[0] is not 0:
            vec = np.mean(lemmas_vectors, axis=0)
        return vec

    def fit(self):
        logging.info('Wait: indexing of fasttext')
        matrix_fasttext = np.zeros((len(self.collection), self.model.vector_size))
        start_time = time()
        for row, query in enumerate(self.collection):
            vector = self.build_vec(query.split()) # наш корпус уже нормализован
            for idx, cell in enumerate(vector):
                matrix_fasttext[row][idx] = cell
        logging.info(f'Indexing FastTextSearch takes {time() - start_time} sec')
        if not os.path.isdir(os.path.join(os.path.abspath, 'models')):
            os.mkdir('models')
        np.save('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/models/fasttext.npy', matrix_fasttext)
        return matrix_fasttext

    def build_test_data(self):
        logging.info('Building Test Data')
        if os.path.isfile('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/data/test_data_fasttext.npy'):
            test_data = np.load('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/data/test_data_fasttext.npy', allow_pickle=True)
        else:
            test_data = collections.defaultdict()
            for idx, question in enumerate(self.corpus['question1_n']):
              if self.corpus['is_duplicate'][idx] == 1:
                test_data[idx] = self.build_vec(question)
            np.save('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/data/test_data_fasttext.npy', test_data)
        logging.info('Test Data ready!')
        return test_data

    def measure_accuracy(self, corpus_size=200):
        accuracy = 0
        znamenatel = 0
        for idx, query in enumerate(self.corpus['question1_n'][:corpus_size]):
            if self.corpus['is_duplicate'][idx] == 1:
                res = [x[0] for x in self.search(query)]
                accuracy += int(idx in res) 
                znamenatel += 1
        return accuracy / znamenatel

    def search(self, query):
        vec = self.build_vec(query)
        res = cosine_similarity([vec], self.matrix)
        docs = [(idx, doc) for idx, doc in enumerate(res[0])]
        docs = sorted(docs, key=lambda x: x[1], reverse=True)
        docs = [(x[0], x[1], self.corpus['question2'][x[0]]) for x in docs[:5]]
        return docs

In [16]:
# urllib.request.urlretrieve("http://vectors.nlpl.eu/repository/11/181.zip", "/content/drive/My Drive/Colab Notebooks/infosearch/181.zip")
# z = zipfile.ZipFile('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/181.zip', 'r')
# z.extractall('/content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/')
    
elmo_path = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/elmo'
corpus = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/data/collection.csv'
model = '/content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model'
fts = FastTextSearch(model, corpus)
fts.search('зарплата Индии')


====
Start FastText initializing!
====
Start FastText initializing!
====
Start FastText initializing!
loading Word2VecKeyedVectors object from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model
loading Word2VecKeyedVectors object from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model
loading Word2VecKeyedVectors object from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


loading vectors_vocab from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model.vectors_vocab.npy with mmap=None
loading vectors_vocab from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model.vectors_vocab.npy with mmap=None
loading vectors_vocab from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model.vectors_vocab.npy with mmap=None
loading vectors_ngrams from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model.vectors_ngrams.npy with mmap=None
loading vectors_ngrams from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model.vectors_ngrams.npy with mmap=None
loading vectors_ngrams from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model.vectors_ngrams.npy with mmap=None
loading vectors from /content/drive/My Drive/Colab Notebooks/infosearch/sem3/fasttext/model.model.vectors.npy with mmap=None
loading vectors from /content/drive/My Drive/Co

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


[(147948,
  0.8858306342735927,
  'сколько стоит 100000 зарплат в США, эквивалентное зарплате в Индии'),
 (171261,
  0.8858306342735927,
  'сколько стоит 100000 зарплат в США, эквивалентное зарплате в Индии'),
 (147445, 0.8675609391773177, 'vfx зарплата сингапур'),
 (1072, 0.8484526237490428, 'какова зарплата врача в Индии'),
 (78857, 0.8484526237490428, 'какова зарплата врача в Индии')]

In [17]:
fts.measure_accuracy(50)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:55: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
/usr/local

0.375

# **ELMO**

In [0]:
import tensorflow as tf
from elmo_helpers import tokenize, get_elmo_vectors, load_elmo_embeddings

In [0]:
class ElmoSearch:
  
  def __init__(self):
    tf.reset_default_graph()
    self.batcher, self.sentence_character_ids, self.elmo_sentence_input = \
    load_elmo_embeddings(elmo_path)
    self.vectors = []
    self.collection = []
    
  def build_vec(self, sentences):
     with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        elmo_vectors = get_elmo_vectors(
            sess, sentences, self.batcher, self.sentence_character_ids,
            self.elmo_sentence_input)
        
        results = []
        for vect, sent in zip(elmo_vectors, sentences):
          results.append(np.mean(vect[:len(sent), :], axis=0))
        
        return results
    
  def indexing(self, sentences):
    self.collection = sentences
    with tf.Session() as sess:
      sess.run(tf.global_variables_initializer())
      start = time()
      sentences = [tokenize(sent) for sent in sentences]
      for sent in sentences:
        sent_vec = self.build_vec([sent])
        self.vectors.append(sent_vec[0])
        
      print(f'=====\n'\
            f'ElmoSearch Indexing takes {time() - start} sec'\
            f'for {len(sentences)} docs')
      return self.vectors
    
  def simple_preproc(self, query):
      return [pmm.normal_forms(word)[0] for word in nltk.word_tokenize(query)]
    
  def elmo_search(self, query):
    query = self.simple_preproc(query)
    query_vec = self.build_vec([query])
    results = []
    for idx, doc in enumerate(self.collection):
      results.append((idx, spatial.distance.cosine(self.vectors[idx], query_vec), doc))
    results = sorted(results, key=lambda x: x[1], reverse=True)
    
    return results
  
  def measure_accuracy(self, test_data, number=100, top=5):
    queries = test_data[:number]
    accuracy = 0
    for q1 in queries:
      res = [x[0] for x in self.elmo_search(q1[1])[:top]]
      accuracy += int(q1[0] in res)
    accuracy = "{:.0%}".format(accuracy / number)
    return accuracy
    

In [0]:
els = ElmoSearch()
els.indexing(pd.read_csv(collection)['question2'][:200])

Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
S

=====
ElmoSearch Indexing takes 424.3839144706726 secfor 200 docs


[array([ 0.4799759 ,  0.5444037 , -0.73717874, ...,  0.35052934,
         0.00076787, -0.10136896], dtype=float32),
 array([-0.13440749,  0.53458875, -0.8851584 , ...,  0.17079997,
        -0.08719695, -0.05251867], dtype=float32),
 array([ 0.46714488,  0.07270581, -0.36679628, ...,  0.45913887,
        -0.694062  , -0.0927758 ], dtype=float32),
 array([ 0.49760342,  0.544155  , -0.6765515 , ...,  0.30531082,
         0.08121888,  0.31734103], dtype=float32),
 array([ 0.7912502 , -0.10166163, -0.72437024, ...,  0.4951693 ,
        -0.37494296, -0.259974  ], dtype=float32),
 array([ 0.14698066,  0.05324822, -0.69836587, ..., -0.551923  ,
        -0.6382584 , -0.19895436], dtype=float32),
 array([ 0.68795085, -0.21769005, -0.76855093, ...,  0.40106335,
        -0.24924561, -0.06053931], dtype=float32),
 array([ 0.5813393 , -0.1931304 , -0.640188  , ..., -0.09111083,
        -0.27869183, -0.1581597 ], dtype=float32),
 array([ 0.44731018,  0.1960752 , -0.65031075, ..., -0.14211048,
       

*Пример запроса*

In [0]:
els.elmo_search('зарплаты Индии')

Sentences in this batch: 1


[(55,
  0.8657055348157883,
  'это мой парень который лгать о свой истинный чувство к свой друг и он тайно привлекать к она'),
 (48, 0.8570665568113327, 'как подружиться'),
 (91,
  0.8544826805591583,
  'какой хороший романтический фильм вы когда-либо видеть'),
 (196, 0.8498908877372742, 'есть ли какой-нибудь фильм на телевидение'),
 (135,
  0.8417142778635025,
  'иметь ли значение являться ли человечество зло или нет'),
 (116,
  0.832378476858139,
  'как winston и julia отличаться в роман 1984 как они похожий'),
 (168,
  0.8229560106992722,
  'почему эмоционально оскорбительный человек в мой жизнь несколько бывший друг друг и прошлое коллега босс быть обидчик я они привлекать'),
 (113,
  0.822862446308136,
  'худой ли использовать кто-то или быть материалистичный'),
 (166, 0.8218187093734741, 'что правильно говорить или говорить с'),
 (7, 0.8191768079996109, 'когда вы использовать вместо'),
 (65,
  0.8190822452306747,
  'какой самый важный книга который вы когда-либо читать'),
 (114, 

In [0]:
test_data = build_test_data(corpus, collection)

In [0]:
elmo_accuracy = els.measure_accuracy(test_data=test_data,number=100)

Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
Sentences in this batch: 1
S

In [0]:
elmo_accuracy

'0%'

## Задание

Реализуйте поиск по [Quora question pairs](https://www.kaggle.com/loopdigga/quora-question-pairs-russian) на нескольких векторных моделях

    1. fasttext, модель ruscorpora_none_fasttextskipgram_300_2_2019
    2. elmo, модель ruwikiruscorpora_lemmas_elmo_1024_2019
    3. bert*, RuBERT - необязательно
   
Первые две обученные модели можно скачать на сайте [rusvectores](https://rusvectores.org/en/models/).

BERT делать необязательно, но если сделаете, 6 за курс у вас автоматом. Модель можно [найти тут](http://docs.deeppavlov.ai/en/master/features/models/bert.html).

### __Задача 1__:    
Сравните время индексации корпуса для каждой модели 

### __Задача 2__:    
Выведите качество поиска для каждой модели +  BM25 для сравнения

Качество оцениваем так же, как в прошлом задании:
    - если в топ-5 результатов выдачи попал хоть один релевантный документ, выдача точная
    - если в топ-5 нет ни одного релеватного документа, выдача получает 0
   